# 패키지로딩

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# 데이터 로딩 및 확인

In [10]:
df_airport = pd.read_csv("./데이터셋/Airport Codes Dataset.csv")
print(df_airport.shape)
display(df_airport.head())

(365, 4)


,airport_id,city,state,name
0,10165,Adak Island,AK,Adak
1,10299,Anchorage,AK,Ted Stevens Anchorage International
2,10304,Aniak,AK,Aniak Airport
3,10754,Barrow,AK,Wiley Post/Will Rogers Memorial
4,10551,Bethel,AK,Bethel Airport


In [11]:
df_flight = pd.read_csv("./데이터셋/Flight on-time performance.csv")
print(df_flight.shape)
display(df_flight.head())

(504397, 18)


,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepTimeBlk,DepDelay,DepDel15,CRSArrTime,ArrTimeBlk,ArrDelay,ArrDel15,Cancelled,Diverted
0,2011,4,10,6,4,WN,13495,12191,1435,1400-1459,2.0,0.0,1550,1500-1559,-6.0,0.0,0,0
1,2011,4,10,6,4,WN,13495,12191,1330,1300-1359,-4.0,0.0,1445,1400-1459,-12.0,0.0,0,0
2,2011,4,10,6,4,WN,13495,12191,1030,1000-1059,-2.0,0.0,1145,1100-1159,-14.0,0.0,0,0
3,2011,4,10,6,4,WN,13495,12889,1900,1900-1959,0.0,0.0,2055,2000-2059,-6.0,0.0,0,0
4,2011,4,10,6,4,WN,13495,12889,1340,1300-1359,-1.0,0.0,1530,1500-1559,2.0,0.0,0,0


In [14]:
print(df_flight.isna().sum())
print(df_airport.isna().sum())

Year                  0
Quarter               0
Month                 0
DayofMonth            0
DayOfWeek             0
Carrier               0
OriginAirportID       0
DestAirportID         0
CRSDepTime            0
DepTimeBlk            0
DepDelay           3697
DepDel15           3697
CRSArrTime            0
ArrTimeBlk            0
ArrDelay           4717
ArrDel15           4717
Cancelled             0
Diverted              0
dtype: int64
airport_id    0
city          0
state         0
name          0
dtype: int64


# 데이터 전처리

## 공항정보 데이터의 컬럼 이름을 출발공항으로 변경

In [15]:
df_ori_airport = df_airport.rename(columns={'city':'Ori-city','state':'Ori-state','name':'Ori-airport'})
display(df_ori_airport.head())

,airport_id,Ori-city,Ori-state,Ori-airport
0,10165,Adak Island,AK,Adak
1,10299,Anchorage,AK,Ted Stevens Anchorage International
2,10304,Aniak,AK,Aniak Airport
3,10754,Barrow,AK,Wiley Post/Will Rogers Memorial
4,10551,Bethel,AK,Bethel Airport


## 비행정보 데이터와 출발공항 정보 데이터 병합

In [17]:
merged_df1 = pd.merge(df_flight, df_ori_airport,left_on='OriginAirportID',right_on='airport_id')
# 중복된 출발공항 id 열 삭제
merged_df1.drop ( 'airport_id', axis=1, inplace=True)

In [20]:
df_dest_airport = df_airport.rename(columns={'city':'Dest-city','state':'Dest-state','name':'Dest-airport'})
# 1차 병합된 비행정보 데이터와 도착공항 정보데이터 병합
merged_df2 = pd.merge(merged_df1,df_dest_airport,left_on='DestAirportID',right_on='airport_id')
print(merged_df2.head())
merged_df2.drop ('airport_id', axis=1, inplace=True)
df2.columns

   Year  Quarter  Month  DayofMonth  DayOfWeek Carrier  OriginAirportID  \
0  2011        4     10           6          4      WN            13495   
1  2011        4     10           6          4      WN            13495   
2  2011        4     10           6          4      WN            13495   
3  2011        4     10           7          5      WN            13495   
4  2011        4     10           7          5      WN            13495   

   DestAirportID  CRSDepTime DepTimeBlk  ...  ArrDel15  Cancelled  Diverted  \
0          12191        1435  1400-1459  ...       0.0          0         0   
1          12191        1330  1300-1359  ...       0.0          0         0   
2          12191        1030  1000-1059  ...       0.0          0         0   
3          12191         820  0800-0859  ...       0.0          0         0   
4          12191        1205  1200-1259  ...       0.0          0         0   

      Ori-city  Ori-state                                 Ori-airport  \
0

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier',
       'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'DepTimeBlk',
       'DepDelay', 'DepDel15', 'CRSArrTime', 'ArrTimeBlk', 'ArrDelay',
       'ArrDel15', 'Cancelled', 'Diverted'],
      dtype='object')

In [21]:
df_flight_info = merged_df2[['DayOfWeek', 'Carrier','DepTimeBlk','DepDelay', 'DepDel15','ArrTimeBlk', 'ArrDelay','ArrDel15','Ori-city','Ori-state','Ori-airport','Dest-city' ,'Dest-state','Dest-airport']]

In [29]:
df_flight_info = df_flight_info.dropna()

## 범주형 데이터 인코딩

In [30]:
for col in [col for col in df_flight_info.columns if df_flight_info[col].dtype == 'object']:
    print(f'컬럼명: {col:<15s}\t유니크 값 개수:{df_flight_info[col].unique().size}')

## 라벨인코딩

In [31]:
encoder = LabelEncoder()
df_flight_info = df_flight_info.apply(lambda col : encoder.fit_transform(col) if col.dtype == object else col)

## 데이터 분할

In [34]:
x = df_flight_info.drop('ArrDel15',axis=1)
y = df_flight_info['ArrDel15']

x_train, x_test, y_train, y_test = train_test_split ( x,y,train_size=0.7,random_state=0, stratify=y)

## 학습 및 평가

In [35]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [41]:
y_hat = model.predict(x_test)
print(accuracy_score(y_test,y_hat)) # 정확도

1.0


# Features Importance

In [40]:
importance = {k:v for k,v in zip(x.columns, model.feature_importances_)}
df_importance = pd.DataFrame(pd.Series(importance), columns = ['Importance']).sort_values('Importance', ascending=False)


## 학습 및 평가

In [ ]:
model =RandomForestClassifier()
model.fit(x_train, y_train)
print(accuracy_score(y_test,y_hat))

In [ ]:
model = LogisticRegression(solver = 'sag', max_iter=2000)
model.fit(x_train, y_train)
print(accuracy_score(y_test,y_hat))

# 회귀계수 가중치

In [ ]:
weight = {k:v for k,v in zip(x.columns, model.coef_)}
df_weight = pd.DataFrame(pd.Series(weight), columns=['가중치']).sort_values('가중치', ascending =False)
display(df_weight)                 